In [1]:
pip install Biopython

     |████████████████████████████████| 2.3MB 7.1MB/s 


In [3]:
from google.colab import files
uploaded = files.upload()

Saving test.fasta to test.fasta


**1. SeqIO**

parse the DNA data in fasta format

In [4]:
from Bio import SeqIO
record = SeqIO.read("test.fasta", format="fasta")
print(record)

ID: sequence_unknown
Name: sequence_unknown
Description: sequence_unknown
Number of features: 0
Seq('TGGGCCTCATATTTATCCTATATACCATGTTCGTATGGTGGCGCGATGTTCTAC...GAG')


**2. Blast**

connect with NCBI to execute qBlast and parse the results

In [ ]:
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
result_handle = NCBIWWW.qblast("blastn", "nt", record.format("fasta"))
blast_record = NCBIXML.read(result_handle)

In [ ]:
E_Value_Thresh = 0.01
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_Value_Thresh:
            print('***Alignment***')
            print('sequence:', alignment.title)
            print('length:', alignment.length)
            print('e value:', hsp.expect)
            print(hsp.query)
            print(hsp.match)
            print(hsp.sbjct)

**3. DNA sequence transcribition, translation, and reverse complement strand.**

In [5]:
from Bio.Seq import Seq
my_seq= Seq('ATGGGCCTCATATTTATCCTATATACCATGTTCGTATGGTGGCGCGATGTTCTACGTGAATCCACGTTCGAAGGACATCATACCAAAGTCGTACAATTAGGACCTCGATATGGTTTTATTCTGTTTATCGTATCGGAGGTTATGTTCTTTTTTGCTCTTTTTCGGGCTTCTTCTCATTCTTCTTTGGCACCTACGGTAGAG')
my_rna = my_seq.transcribe()
my_peptide = my_seq.translate()
my_complement = my_seq.reverse_complement()

print('reverse complement is %s' %my_complement)


reverse complement is CTCTACCGTAGGTGCCAAAGAAGAATGAGAAGAAGCCCGAAAAAGAGCAAAAAAGAACATAACCTCCGATACGATAAACAGAATAAAACCATATCGAGGTCCTAATTGTACGACTTTGGTATGATGTCCTTCGAACGTGGATTCACGTAGAACATCGCGCCACCATACGAACATGGTATATAGGATAAATATGAGGCCCAT


**4. Compute GC content**

In [6]:
def gc(dna):
  nbases = dna.count('n')+ dna.count('N')
  gcpercent=float(dna.count('C')+dna.count('c')+dna.count('G')+dna.count('g')) *100.0/ (len(dna)-nbases)
  
  return gcpercent

seq_gc= gc(my_seq)
print('The gc content is {:0.2f}% '.format(seq_gc))

The gc content is 41.79% 


**5. Find the longest ORF for each read**

In [7]:
from google.colab import files
uploaded = files.upload()

Saving dna2.fasta to dna2.fasta


In [8]:
from Bio import SeqIO
records = list(SeqIO.parse("dna2.fasta", "fasta"))
print(records)

[SeqRecord(seq=Seq('CTCGCGTTGCAGGCCGGCGTGTCGCGCAACGACGTGTGGGGCCTGACGGGCAGG...GCC'), id='gi|142022655|gb|EQ086233.1|91', name='gi|142022655|gb|EQ086233.1|91', description='gi|142022655|gb|EQ086233.1|91 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence', dbxrefs=[]), SeqRecord(seq=Seq('CGCGATCCAGTAGCGCTTGTAGCCGAGCGCTTCGGCACGCTTCGCGAGCGCGAT...GCC'), id='gi|142022655|gb|EQ086233.1|304', name='gi|142022655|gb|EQ086233.1|304', description='gi|142022655|gb|EQ086233.1|304 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence', dbxrefs=[]), SeqRecord(seq=Seq('CTCGACGCGCTCCGCGTCGAGGTCGCCCGACGTCTCGCGCAGCAACTGATTCAA...TCG'), id='gi|142022655|gb|EQ086233.1|255', name='gi|142022655|gb|EQ086233.1|255', description='gi|142022655|gb|EQ086233.1|255 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence', dbxrefs=[]), SeqRecord(seq=Seq('CGTGCTCGGCACGACTATCAGCCCGTATCTGTTTTTCTGGCAGGCCTCCCAGGA...CC

In [ ]:
for i in range(len(records)):
    print(i,len(records[i].seq), records[i].seq)

In [38]:
def findORF(records,rf=0):  #records = DNA reads; rf= 0, 1, or 2(different readingframe)
    startCodon = 'ATG'
    stopCodon = ['TAA','TAG','TGA']
    results={}

    for i in range(len(records)):                      #read sequence one by one
        test_seq = records[i].seq
        longestLength = 0
        startPosition= 0; stopPosition=0
        #test_seq = test_seq.reverse_complement()  #finding orf in reverse_complement strand

        for j in range(rf,len(test_seq),3):  #finding start codon
            if test_seq[j:j+3] == startCodon:
                startPosition = j
            
                for k in range(j+3,len(test_seq),3):
                    if test_seq[k:k+3] in stopCodon:    #after finding start and end, slice out orf
                        stopPosition = k
                        orf = test_seq[startPosition:stopPosition+3]

                        if len(orf) > longestLength:  #recording the longest orf, its starting position, and its length
                            longestLength= len(orf)
                            llstart = startPosition
                            results[records[i].id] = [llstart,longestLength]
                        startPosition= 0; stopPosition=0
                        break
    return results

In [39]:
my_results = findORF(records,1)
for k,v in my_results.items():
    print(k,' Starting positoin:',v[0],' Longest length',v[1])


gi|142022655|gb|EQ086233.1|91  Starting positoin: 817  Longest length 237
gi|142022655|gb|EQ086233.1|255  Starting positoin: 430  Longest length 1185
gi|142022655|gb|EQ086233.1|45  Starting positoin: 157  Longest length 528
gi|142022655|gb|EQ086233.1|396  Starting positoin: 1516  Longest length 1281
gi|142022655|gb|EQ086233.1|250  Starting positoin: 163  Longest length 552
gi|142022655|gb|EQ086233.1|322  Starting positoin: 274  Longest length 156
gi|142022655|gb|EQ086233.1|88  Starting positoin: 181  Longest length 138
gi|142022655|gb|EQ086233.1|594  Starting positoin: 694  Longest length 33
gi|142022655|gb|EQ086233.1|293  Starting positoin: 1573  Longest length 420
gi|142022655|gb|EQ086233.1|75  Starting positoin: 256  Longest length 504
gi|142022655|gb|EQ086233.1|454  Starting positoin: 757  Longest length 822
gi|142022655|gb|EQ086233.1|16  Starting positoin: 3070  Longest length 1458
gi|142022655|gb|EQ086233.1|584  Starting positoin: 202  Longest length 27
gi|142022655|gb|EQ086233.1